# Integration tests for the GoFigr Python client

## Lite version

In [ ]:
import json
from platform import platform
from collections import defaultdict

import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
from uuid import uuid4
from gofigr._version import VERSION

import plotly
import plotly.graph_objects as go
import PIL


from datetime import datetime

START_TIME = datetime.now()
TEST_RESULTS = []
LAST_FIGURE = None

def sync():
    with open("lite_tests.json", "w") as f:
        json.dump({
            'platform': platform(),
            'elapsed_seconds': (datetime.now() - START_TIME).total_seconds(),
            'results': TEST_RESULTS
            }, f, indent=4)

def check_text(txt, min_len=10):
    return txt is not None and txt.strip() not in ["N/A", "NA"] and len(txt.strip()) >= min_len

def check_figure(test_name, fig_name, expected_revisions=1):  
    global LAST_FIGURE
    
    res = defaultdict(lambda: True)
    res["test_name"] = test_name
    res["error"] = None
    res["elapsed_seconds"] = (datetime.now() - START_TIME).total_seconds()

    try:
        ext = get_extension()
        if len(ext.publisher.revision_log) == 0:
            res['figure_processed'] = False
        else:
            new_fig = ext.publisher.revision_log[-1]
            res['figure_processed'] = new_fig is not LAST_FIGURE
            LAST_FIGURE = new_fig

            meta = new_fig.revision.metadata
            res['comm'] = ext.comm_data is not None
            res['extension_version'] = ext.comm_data['extension_version'] == VERSION
            res['metadata'] = meta is not None
            res['notebook_name'] = meta['notebook_name'] == "lite_tests.ipynb"
            res['notebook_path'] = os.path.exists(meta['notebook_path'])
            res['title'] = new_fig.title == fig_name
            res['image_not_null'] = new_fig.displayed_image is not None
            res['image_type'] = type(new_fig.displayed_image).__name__
            if "plotly" in test_name.lower():
                res['image_type_check'] = isinstance(new_fig.displayed_image, go.Figure)
            else:
                res['image_type_check'] = isinstance(new_fig.displayed_image, PIL.Image.Image)
                
    except Exception as e:
        print(e)
        res["error"] = str(e)
        
            
    return dict(res.items())

# MPL: load the extension and publish a figure from the same cell

In [ ]:
%load_ext gofigr.lite

get_extension().publisher.log_revisions = True  # store revisions

sns.set_style("darkgrid")
sns.set_context("talk") 

# Try plotting in the same cell as configure(). auto_publish will break, but we should at least
# get a warning
fig_title1 = str(uuid4())
plt.figure()
plt.scatter(range(10), range(10))
plt.title(fig_title1)

In [ ]:
TEST_RESULTS.append(check_figure("MPL: Configure and publish in one cell", fig_title1))
sync()

In [ ]:
TEST_RESULTS

# MPL: Anonymous figure

In [ ]:
plt.scatter(x=range(10), y=range(10))

In [ ]:
TEST_RESULTS.append(check_figure("MPL: Anonymous figure", None))
sync()

# MPL: Manual call to publish

In [ ]:
iris = sns.load_dataset("iris")

title2 = str(uuid4())

plt.figure(figsize=(6, 6))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="tab10")
plt.title(title2)

publish(dataframes={"IRIS": iris})

In [ ]:
TEST_RESULTS.append(check_figure("MPL: Manual publish", title2))
sync()

# MPL: Auto publish

In [ ]:
iris = sns.load_dataset("iris")

title3 = str(uuid4())
plt.figure(figsize=(6, 6))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="tab10")
plt.title(title3)

In [ ]:
TEST_RESULTS.append(check_figure("MPL: Auto publish", title3))
sync()

# MPL: Add revision

In [ ]:
# Add a new revision to the same figure
plt.figure(figsize=(6, 6))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="tab10")
plt.title(title3)

In [ ]:
TEST_RESULTS.append(check_figure("MPL: Auto-publish second revision", title3,
                                expected_revisions=2))
sync()

# MPL: Inline backend

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format="retina"

title4 = str(uuid4())
plt.figure()
plt.scatter(range(10), range(10))
plt.title(title4)

In [ ]:
TEST_RESULTS.append(check_figure("MPL: inline backend", title4))
sync()

# MPL: Mix-and-match manual and auto publish

In [ ]:
title5 = str(uuid4())
plt.figure(figsize=(6, 6))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="tab10")
plt.title(title5)
publish()

plt.figure(figsize=(4, 4))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="Set1")
plt.title(title5)

In [ ]:
TEST_RESULTS.append(check_figure("MPL: Mix manual and auto-publish", title5, expected_revisions=2))
sync()

# MPL: Subplots

In [ ]:
import numpy as np

title6 = str(uuid4())
fig, ax = plt.subplots(nrows=2, ncols=2)
ax = np.ravel(ax)

for idx, axis in enumerate(ax):
    axis.scatter(x=np.random.normal(loc=idx, size=100), y=np.random.normal(loc=idx, size=100))

fig.suptitle(title6)

In [ ]:
TEST_RESULTS.append(check_figure("MPL: Subplots", title6, expected_revisions=1))
sync()

# Plotly

## Static figure

In [ ]:
import numpy as np
np.random.seed(1)

title7 = str(uuid4())

N = 100
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
sz = np.random.rand(N) * 30

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode="markers",
    marker=go.scatter.Marker(
        size=sz,
        color=colors,
        opacity=0.6,
        colorscale="Viridis"
    )))
fig.update_layout(title=title7)

fig.show()

publish(fig=fig, image_options={'width': 800, 'height': 600})
publish(fig=fig, image_options={'width': 1000, 'height': 800})

In [ ]:
TEST_RESULTS.append(check_figure("Plotly - manual publish", title7, expected_revisions=3))
sync()

## Plotly -- autopublish

In [ ]:
import plotly.express as px

title8 = str(uuid4())
df = px.data.iris()
fig = px.scatter(df, x="sepal_length", y="sepal_width", color="species",
                title=title8)
fig.show()

In [ ]:
TEST_RESULTS.append(check_figure("Plotly - autopublish", title8, expected_revisions=1))
sync()

# plotnine - basic plot

In [ ]:
from plotnine import *
from plotnine.data import *

In [ ]:
title13 = str(uuid4())

fig = (
    ggplot(mtcars, aes("wt", "mpg", color="factor(gear)"))
    + geom_point()
    + stat_smooth(method="lm")
    + facet_wrap("gear")
    + ggtitle(title13)
)

fig

In [ ]:
TEST_RESULTS.append(check_figure("plotnine - basic figure", title13, expected_revisions=1))
sync()

# plotnine - facets

In [ ]:
meat_long = meat.melt(
    id_vars="date",
    value_vars=["beef", "veal", "pork", "lamb_and_mutton", "broilers", "turkey"],
    var_name="animal",
    value_name="weight"
).dropna()

meat_long.head()

In [ ]:
title14 = str(uuid4())

# Gallery, lines
def titled(strip_title):
    return " ".join(s.title() if s != "and" else s for s in strip_title.split("_"))

(
    ggplot(meat_long, aes("date", "weight", color="animal"))
    + geom_line(size=.5, show_legend=False)
    + facet_wrap("animal", labeller=titled)
    + scale_x_datetime(date_breaks="20 years", date_labels="%Y")
    + labs(
        x="Date",
        y="Weight (million pounds)",
        title=title14,
    )
    + theme_538(base_size=9)
)

In [ ]:
TEST_RESULTS.append(check_figure("plotnine - facets", title14, expected_revisions=1))
sync()

# plotnine - manual publish

In [ ]:
title15 = str(uuid4())

fig = (
    ggplot(mtcars, aes("wt", "mpg", color="factor(gear)"))
    + geom_point()
    + stat_smooth(method="lm")
    + facet_wrap("gear")
    + ggtitle(title15)
)

In [ ]:
publish(fig)

In [ ]:
TEST_RESULTS.append(check_figure("plotnine - manual publish", title15, expected_revisions=1))
sync()

# View results

In [ ]:
import pandas as pd
pd.DataFrame(TEST_RESULTS)

# Save results

In [ ]:
with open("integration_test.json.done", "w") as f:
    f.write("done\n")

In [ ]:
!pwd

In [ ]:
print(f"Finished in {datetime.now() - START_TIME}")

In [ ]:
get_extension